# Generate piece wise audio files

One second pre audio files for each piece to one second past onset of claps? or end of song? lets See.


https://github.com/jiaaro/pydub


In [3]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json 
from scipy.signal import butter, filtfilt, argrelextrema
from scipy import interpolate
from scipy.interpolate import interp1d

from pydub import AudioSegment

In [4]:
# audio file locations in
audio_loc = '../Data/Audio/Full_Zero/'
out_loc = '../Data/Audio/Piece_wise/'
# audio file locations out

Piece_timings = pd.read_csv('./Timing/Concerttimes_Seconds.csv')


Index(['Pre-Audio', 'Warmup', 'Tuning', 'Synch Taps', 'Applause for Conductor',
       'Saevarud', 'Applause for Saevarud', 'MC pre Reich', 'Reich',
       'Applause for Reich', 'MC pre Arban', 'Applause for Florian', 'Arban',
       'Applause for Arban', 'MC pre Grieg', 'Grieg', 'Applause for Grieg',
       'MC pre Tsjajkovskij', 'Tsjajkovskij', 'Applause for Tsjajkovskij',
       'MC pre Christophersen', 'Christophersen',
       'Applause for Christophersen', 'MC pre School',
       'Applause for school class', 'MC pre Vivaldi', 'Vivaldi',
       'Applause for Vivaldi', 'MC post Vivaldi', 'Applause for school class',
       'MC pre Strauss', 'Strauss', 'Applause for Strauss', 'End'],
      dtype='object', name='Event')


,RD,C1,C2,C3,C4,C5
Event,,,,,,
Pre-Audio,NaN,NaN,-421.378,NaN,NaN,-981.902
Warmup,-300.000,-348.482,-361.303,-52.946,-344.999,-386.462
Tuning,-58.745,-61.511,-49.122,-48.773,-48.983,-225.636
Synch Taps,0.000,0.000,0.000,0.000,0.000,0.000
Applause for Conductor,19.294,28.292,122.014,76.074,21.929,-30.137
Saevarud,43.907,44.900,3279.222,3288.046,30.147,15.336
Applause for Saevarud,280.462,279.156,3511.465,3518.249,265.054,246.166
MC pre Reich,285.523,294.284,268.682,218.914,274.913,257.826
Reich,673.822,723.922,648.230,579.166,653.881,694.972


In [43]:
Piece_timings = pd.read_csv('./Timing/Concerttimes_Seconds.csv')


In [44]:
datafile_type = 'Audio'
concert = 'C5'
piece = 'Full'
audio_loc = '../Data/Audio/Full_Zero/' + concert + '_Full_Zero.mp3'
out_loc = '../Data/Audio/Piece_wise/'
C_times = Piece_timings[['Event','Event_Type','Related']].copy()
C_times['Start'] =  Piece_timings[concert]
C_times = C_times.sort_values('Start')
C_times = C_times.loc[C_times['Start']>-15,:].reset_index(drop = True).copy()
C_times['Next'] = C_times.Start.shift(-1)
M_times =  C_times.loc[C_times['Event_Type'] == 'Music',:]
M_times

,Event,Event_Type,Related,Start,Next
1,Saevarud,Music,Saev,15.336,246.166
4,Reich,Music,Reic,694.972,802.967
8,Arban,Music,Arba,1012.486,1520.986
11,Grieg,Music,Grie,1581.658,1904.608
14,Tsjajkovskij,Music,Tcha,2102.386,2273.980
17,Christophersen,Music,Chri,2362.576,2739.538
22,Vivaldi,Music,Viva,3008.884,3242.818
27,Strauss,Music,Stra,3441.964,3564.004


In [45]:
sound = AudioSegment.from_mp3(audio_loc)
print(len(sound))

3602888


In [37]:
# buffer is small, 1 second
t_units = 1000  # pydub does things in milliseconds
buff = 1 # in seconds
for i in M_times.index:
    Peice_tag = M_times.loc[i,'Related']
    s_time = (M_times.loc[i,'Start']-buff) * t_units
    e_time = (M_times.loc[i,'Next']+buff) * t_units
    p_audio = sound[s_time:e_time]
    out_audio = out_loc + concert + '_' + Peice_tag + '_1.mp3'
    with open(out_audio, 'wb') as out_f:
        p_audio.export(out_f, format='mp3')

## get pre Arban audio

In [46]:
 Piece_timings.loc[Piece_timings['Related'] == 'Arba']

,Event,Event_Type,Related,RD,C1,C2,C3,C4,C5
10,MC pre Arban,Speech,Arba,792.286,842.650,773.062,700.660,780.809,813.700
11,Applause for Florian,Applause,Arba,835.470,963.502,907.786,929.872,1004.009,997.015
12,Arban,Music,Arba,860.595,982.114,924.186,948.014,1045.313,1012.486
13,Applause for Arban,Applause,Arba,1382.782,1505.664,1453.954,1452.526,1538.147,1520.986


In [47]:
buff = 1
out_loc = '../Data/Audio/Piece_wise/'

for concert in ['C1','C2','C3','C4','C5']:
    audio_loc = '../Data/Audio/Full_Zero/' + concert + '_Full_Zero.mp3'
    sound = AudioSegment.from_mp3(audio_loc)
    s_time = (Piece_timings.loc[10,concert]-buff) * t_units
    e_time = (Piece_timings.loc[11,concert]+buff) * t_units
    p_audio = sound[s_time:e_time]
    out_audio = out_loc + concert + '_MC_Pre_Arban_1.mp3'
    with open(out_audio, 'wb') as out_f:
        p_audio.export(out_f, format='mp3')